# VOCS data structure 

Variables, Objectives, Constraints, and other Settings (VOCS) helps define our optimization problems. 

In [1]:
from xopt.vocs import VOCS

In [2]:
help(VOCS)

Help on class VOCS in module xopt.vocs:

class VOCS(xopt.pydantic.XoptBaseModel)
 |  VOCS(*, variables: Dict[str, types.ConstrainedListValue] = {}, constraints: Dict[str, types.ConstrainedListValue] = {}, objectives: Dict[str, xopt.vocs.ObjectiveEnum] = {}, constants: Dict[str, Any] = {}, linked_variables: Dict[str, str] = {}) -> None
 |  
 |  Variables, Objectives, Constraints, and other Settings (VOCS) data structure
 |  to describe optimization problems.
 |  
 |  Method resolution order:
 |      VOCS
 |      xopt.pydantic.XoptBaseModel
 |      pydantic.main.BaseModel
 |      pydantic.utils.Representation
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  as_yaml(self)
 |  
 |  constraint_data(self, data: Union[pandas.core.frame.DataFrame, List[Dict]], prefix: str = 'constraint_') -> pandas.core.frame.DataFrame
 |      Returns a dataframe containing constraint data transformed according to
 |      `vocs.constraints` such that values that satisfy each constraint are negat

In [3]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, linked_variables={})

In [4]:
# as dict
vocs.dict()

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'linked_variables': {}}

In [5]:
#  re-parse dict
vocs2 = VOCS.parse_obj(vocs.dict())

In [6]:
# Check that these are the same
vocs2 == vocs

True

In [7]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [8]:
vocs.objectives["c"] == 'MAXIMIZE'

True

In [9]:
# json
vocs.json()

'{"variables": {"a": [0.0, 1000.0], "b": [-1.0, 1.0]}, "constraints": {"e": ["LESS_THAN", 2.0], "f": ["GREATER_THAN", 0.0]}, "objectives": {"c": "MAXIMIZE", "d": "MINIMIZE"}, "constants": {"g": 1234}, "linked_variables": {}}'

# Objective Evaluation

In [10]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

a         b     g           c           d            e  \
5   933.985689  0.693333  1234  934.679021  933.292356  1869.358043   
6   109.936871 -0.897851  1234  109.039020  110.834722   218.078041   
7   116.967559  0.468390  1234  117.435949  116.499170   234.871899   
8   871.432485 -0.844948  1234  870.587536  872.277433  1741.175073   
9   393.349987 -0.722784  1234  392.627202  394.072771   785.254405   
10  771.166640  0.386022  1234  771.552661  770.780618  1543.105323   
11  519.255957  0.857517  1234  520.113473  518.398440  1040.226947   
12  632.201130  0.749710  1234  632.950839  631.451420  1265.901679   
13  286.821439  0.535515  1234  287.356955  286.285924   574.713909   
14  255.487109  0.456693  1234  255.943802  255.030416   511.887604   

              f  
5   1866.584713  
6    221.669444  
7    232.998339  
8   1744.554865  
9    788.145542  
10  1541.561236  
11  1036.796881  
12  1262.902840  
13   572.571848  
14   510.060832

In [11]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [12]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

objective_c  objective_d
5   -934.679021   933.292356
6   -109.039020   110.834722
7   -117.435949   116.499170
8   -870.587536   872.277433
9   -392.627202   394.072771
10  -771.552661   770.780618
11  -520.113473   518.398440
12  -632.950839   631.451420
13  -287.356955   286.285924
14  -255.943802   255.030416

In [13]:
# This is also available as a method
vocs.objective_data(data)

objective_c  objective_d
5   -934.679021   933.292356
6   -109.039020   110.834722
7   -117.435949   116.499170
8   -870.587536   872.277433
9   -392.627202   394.072771
10  -771.552661   770.780618
11  -520.113473   518.398440
12  -632.950839   631.451420
13  -287.356955   286.285924
14  -255.943802   255.030416

In [14]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[-934.67902134,  933.29235629],
       [-109.03902045,  110.83472211],
       [-117.43594933,  116.49916954],
       [-870.58753642,  872.27743267],
       [-392.62720239,  394.07277101],
       [-771.55266146,  770.78061808],
       [-520.11347343,  518.39844028],
       [-632.9508395 ,  631.45141999],
       [-287.35695459,  286.28592417],
       [-255.94380202,  255.03041579]])

In [15]:
vocs.constraint_data(data)

constraint_e  constraint_f
5    1867.358043  -1866.584713
6     216.078041   -221.669444
7     232.871899   -232.998339
8    1739.175073  -1744.554865
9     783.254405   -788.145542
10   1541.105323  -1541.561236
11   1038.226947  -1036.796881
12   1263.901679  -1262.902840
13    572.713909   -572.571848
14    509.887604   -510.060832

In [16]:
vocs.feasibility_data(data)

feasible_e  feasible_f  feasible
5        False        True     False
6        False        True     False
7        False        True     False
8        False        True     False
9        False        True     False
10       False        True     False
11       False        True     False
12       False        True     False
13       False        True     False
14       False        True     False

# Error handling

In [17]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [18]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [19]:
data['a']  = np.nan

In [20]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [21]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

A
1  6.000000e+66
2  6.000000e+66
3  6.000000e+66

In [22]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

objective_c  objective_d
9         -1.0          inf
3         -2.0          inf
4         -3.0          inf
5         -4.0          inf